# Librerías utilizadas

In [1]:
%pylab
#pylab.rcParams['figure.figsize'] = (10, 6)

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


# Funciones

In [2]:
def GaussianNoise(SNR,Ps,M,N):
    """
    Recibe una potencia de señal Ps y una relación señal-ruido deseada
    SNR y retorna un vector Nu correspondiente a muestras del ruido 
    blanco gaussiano a sumarle a la señal
    """
    mean_W = 0
    var_W = Ps/(10**(SNR/10))
    std_W = np.sqrt(var_W)
    W = np.asmatrix(np.random.normal(mean_W, std_W, size=(M,N)))
    return (W)

def print_matrix(list_of_list):
    number_width = len(str(max([max(i) for i in list_of_list])))
    cols = max(map(len, list_of_list))
    output = '+'+('-'*(number_width+2)+'+')*cols + '\n'
    for row in list_of_list:
        for column in row:
            output += '|' + ' {:^{width}d} '.format(column, width = number_width)
        output+='|\n+'+('-'*(number_width+2)+'+')*cols + '\n'
    return output

# Definición de constantes y unidades

In [3]:
kHz = 10 ** 3
MHz = 10 ** 6
GHz = 10 ** 9
ms = 10 ** -3
c = 3 * 10 ** 8
km = 10 ** 3

# Definición de variables

In [4]:
M = 8  # Número de sensores
D = 3  # Número de señales
N = 100 # Número de snapshots

fs = 64 * MHz

T = 5 * ms # Tiempo de toma de muestras
t0 = 0

theta_deg = np.array([30,90,120])  #DOA en grados
theta = theta_deg * np.pi / 180           #DOA en radianes

# Defición de señal y array de sensores
Fc = np.array([436*MHz, 436*MHz, 436*MHz]) # Frecuencia de portadora de la señal transmitida 1
lambda_c = c / Fc
d = lambda_c[0] / 2  #Separación entre sensores

s_amp = np.array([100,100,100])
s_freq = np.array([440,1000,30000])

SNR = 7

# Definición de señales y muestras

## Definición del vector aleatorio de tiempo de toma de snapshots

In [5]:
t = np.random.randint(0, int(T * fs), size=N) * 1 / fs  # Vector de tiempos de muestreo (tomas de snapshots)

## Genero la matriz F de tamaño (D,N)

In [6]:
lambda_c = c / Fc
K = 2 * np.pi / (lambda_c)
F = np.asmatrix(np.empty((D, N)))
for i in range(0, D):
    F[i,:] = s_amp[i] * np.cos(2 * np.pi * s_freq[i] * (t+t0))

## Genero la matriz A de tamaño (M,D) considerando $g(\theta)=1 \quad \forall \  \theta$

In [7]:
A = np.asmatrix(np.empty((M, D), dtype='complex'))
for i in range(0, M):
    for j in range(0, D):
        A[i, j] = np.e ** (-1j * i * K[j] * d * np.cos(theta[j]))

## Genero el vector W de tamaño (M,N) de ruido a partir de una SNR dada

In [8]:
Ps = 0
for i in range(0,D):
    Ps = Ps + (s_amp[i]**2)/2
W = GaussianNoise(SNR,Ps,M,N)

## Genero el vector de muestras $X=A \times F + W$ de tamaño (M,1,N)

In [9]:
X = np.asmatrix(np.empty((M,N),dtype=complex))
X_matrix = np.asmatrix(np.empty((M,1)),dtype=complex)
for n in range (0,N):
    F_matrix = F[:,n]
    W_matrix = W[:,n]
    X_matrix = A @ F_matrix + W_matrix
    X[:, n] = X_matrix

# Calculo el vector de covarianza S

In [10]:
S = np.asmatrix(np.empty((M,M),dtype=complex))

for n in range (0,N):
    X_matrix=np.asmatrix(X[:,n])
    S = S + (1/N)*(X_matrix @ X_matrix.H)

# Encuentro autovalores y autovectores S y el autovalor mínimo $\lambda_{min}$ tal que $|S-\lambda_{min}\cdot S_0|=0$

In [11]:
[aval, avec] = eig(S)
S0 = np.identity(M)
p = aval.argsort()[::-1]
aval=np.abs(aval[p])
avec=avec[:,p]
aval_min = aval[-1]

In [12]:
figure()
bins_aval=hist(aval,bins=50)
show()

# Encuentro la multiplicidad Q de $\lambda_min$ y el número estimado de señales $\hat{D}$

In [13]:
bins_aval=histogram(aval,bins=50)
for i in range(0,aval.size):
    if bins_aval[0][i] == 0:
        umbral = bins_aval[1][i]
        break

Q = 0

for i in range (0,M):
    if aval[i]<umbral:
        Q += 1
D_est = M - Q

# Formo la matriz de subespacio de ruido $E_N$

In [14]:
EN = np.asmatrix(avec[:,D_est:])

# Calculo la matriz $C = E_N E_N^H$

In [15]:
C = EN @ EN.H

# Encuentro el polinomio Root-MUSIC

In [16]:
Pz = np.zeros(2*M-1,dtype=complex)

for i in range (0,Pz.size):
    Pz[i]=sum(diag(C,k=-M+1+i))

# Encuentro las raíces del polinomio y las DOA

In [27]:
Pz_roots=roots(Pz)
Pz_roots_all = Pz_roots
Pz_roots=Pz_roots[(abs(Pz_roots)<=1)]
Pz_roots = Pz_roots[argsort(abs(Pz_roots))]
Pz_roots = Pz_roots[-D_est:]
theta_est=arccos( angle(Pz_roots) /(K[0]*d) )*180/pi


In [22]:
figure()
plot(real(Pz_roots_all),imag(Pz_roots_all),'o')
plot(real(exp(1j*linspace(0,2*pi,100))),imag(exp(1j*linspace(0,2*pi,100))))
show()

In [29]:
figure()
bins_Pz=hist(abs(Pz_roots_all),bins=50)
show()

# Creo funciones para automatizar la creación de las muestras y el algoritmo music

In [33]:
def doamusic_samples(Fc,s_amp,s_freq,t,t0,d,theta,M,D,SNR):
    # Definición de señales y muestras
    # Genero la matriz F de tamaño (D,N)
    lambda_c = c / Fc
    K = 2 * np.pi / (lambda_c)
    F = np.asmatrix(np.empty((D, N)))
    for i in range(0, D):
        F[i,:] = s_amp[i] * np.cos(2 * np.pi * s_freq[i] * (t+t0))

    # Genero la matriz A de tamaño (M,D) considerando $g(\theta)=1 \quad \forall \  \theta$
    A = np.asmatrix(np.empty((M, D), dtype='complex'))
    for i in range(0, M):
        for j in range(0, D):
            A[i, j] = np.e ** (-1j * i * K[j] * d * np.cos(theta[j]))

    # Genero el vector W de tamaño (M,1,N) de ruido $W\sim N(0,1)$
    Ps = 0
    for i in range(0,D):
        Ps = Ps + (s_amp[i]**2)/2
    W = GaussianNoise(SNR,Ps,M,N)

    # Genero el vector de muestras $X=A \times F + W$ de tamaño (M,1,N)
    X = np.asmatrix(np.empty((M,N),dtype=complex))
    X_matrix = np.asmatrix(np.empty((M,1)),dtype=complex)
    for n in range (0,N):
        F_matrix = F[:,n]
        W_matrix = W[:,n]
        X_matrix = A @ F_matrix + W_matrix
        X[:, n] = X_matrix

    return X

def doa_rootmusic_estimation(X, K_est):
    M = X.shape[0]
    N = X.shape[1]

    #%% Calculo el vector de covarianza S
    S = np.asmatrix(np.empty((M,M),dtype=complex))
    for n in range (0,N):
        X_matrix=np.asmatrix(X[:,n])
        S = S + (1/N)*(X_matrix @ X_matrix.H)

    # Encuentro autovalores y autovectores S y el autovalor mínimo $\lambda_{min}$ tal que $|S-\lambda_{min}\cdot S_0|=0$
    [aval, avec] = eig(S)
    S0 = np.identity(M)
    p = aval.argsort()
    aval=np.abs(aval[p])
    avec=avec[:,p]
    aval_min = aval[0]

    # Encuentro la multiplicidad Q de $\lambda_min$ y el número estimado de señales $\hat{D}$
    bins_aval=histogram(aval,bins=50)
    for i in range(0,aval.size):
        if bins_aval[0][i] == 0:
            umbral = bins_aval[1][i]
            break

    Q = 0

    for i in range (0,M):
        if aval[i]<umbral:
            Q += 1
    D_est = M - Q

    # Formo la matriz de subespacio de ruido $E_N$
    EN = np.asmatrix(avec[:,0:Q])

    # Calculo la matriz $C = E_N E_N^H$
    C = EN @ EN.H

    # Encuentro el polinomio Root-MUSIC
    Pz = np.zeros(2*M-1,dtype=complex)

    for i in range (0,Pz.size):
        Pz[i]=sum(diag(C,k=-M+1+i))

    # Encuentro las raíces del polinomio y las DOA
    Pz_roots = roots(Pz)
    Pz_roots = Pz_roots[(abs(Pz_roots)<=1)]
    Pz_roots = Pz_roots[argsort(abs(Pz_roots))]
    Pz_roots = Pz_roots[-D_est:]
    theta_est = arccos( angle(Pz_roots) /(K_est[0]*d) )*180/pi

    return theta_est

# Script

In [34]:
M = 8  # Número de sensores
D = 3  # Número de señales
N = 1000 # Número de snapshots

fs = 64 * MHz

T = 5 * ms # Tiempo de toma de muestras
t0 = 0

theta_deg = np.array([40,90,120])  #DOA en grados
theta = theta_deg * np.pi / 180           #DOA en radianes

# Defición de señal y array de sensores
Fc = np.array([436.5*MHz, 436*MHz, 435.5*MHz]) # Frecuencia de portadora de la señal transmitida 1
lambda_c = c / Fc
d = lambda_c[0] / 2  #Separación entre sensores

s_amp = np.array([50,100,200])
s_freq = np.array([440,1000,30000])

SNR = 7
t = np.random.randint(0, int(T * fs), size=N) * 1 / fs  # Vector de tiempos de muestreo (tomas de snapshots)
K_est = 2 * np.pi * Fc/c

In [36]:
X = doamusic_samples(Fc, s_amp, s_freq, t,0, d, theta, M, D, SNR)

In [37]:
theta_est = doa_rootmusic_estimation(X, K_est)